The aim of the project is to develop a machine learning model capable of predicting the bioactivity of molecules within the dataset sourced from PubChem, a public database, containing molecules categorised as either "active" or "inactive" against the specified target. 
Firstly, we are going to process, debug and clean the dataset to enhance the dataset quaility. This process includes converting SMILES to Canonical SMILES follwed by salt removal to identify and remove any duplicated molecules or compounds with missleading activity lables. We then use the pre-porcessed dataset to calculate a number of molecular descriptors (e.g., physchem properties, 2D fingerprints etc.). The calculated moleuclar descriptors of the chemical series in the pre-processed dataset will be used as training set to build a machine learning model to predict the bioactivity of the compouands.

In [1]:
import sklearn
import pandas as pd
import matplotlib
import numpy as np
import rdkit as rd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import MACCSkeys
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import SaltRemover
import ipykernel
from IPython.display import HTML
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

Initially, we'll identify any missing values or invalid SMILES within the SMILES column. Subsequently, we'll transform the SMILES representations into molecular structures, creating a new column to visualize these structures as molecular graphs. Finally, we'll verify if there are any missing values present in the newly generated column.

In [3]:
df=pd.read_csv('/var/mlproj/ChemApps/XML/NP_004081-0.0.csv')
df['Molecule']= [Chem.MolFromSmiles(s) for s in df['SMILES']]
print(df.shape)
print(df[['Molecule']].isna().value_counts())
print(df[df['Molecule'].isna()==True].SMILES.tolist())
df.isna().any()

(3288, 3)
Molecule
False       3288
dtype: int64
[]


SMILES      False
Activity    False
Molecule    False
dtype: bool

Canonical SMILES provides a solution for removing duplicate molecules from the dataset by offering a distinct representation of chemical structures. Here, we introduce a function to convert SMILES to canonical SMILES.

In [4]:
def generate_canonical_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    else:
        return "Invalid SMILES"

# call the function
CAN_SMILES = df["Canonical_SMILES"] = df["SMILES"].apply(generate_canonical_smiles)
df = df.reset_index(drop=True)

And after calling the function, we check if there's any duplicate moelcules:

In [5]:

duplicates_indices = df[(df.groupby('Canonical_SMILES').transform('count')>1).values].index.tolist()
duplicates_indices = list(set(duplicates_indices))
print("number of all duplicated cansmiles: ", len(duplicates_indices)) 

number of all duplicated cansmiles:  2


We have identified two duplicated molecules and we can track them now.

In [6]:
is_duplicate_cansmiles = df[df['Canonical_SMILES'].duplicated()==True]  
print(is_duplicate_cansmiles)


                                  SMILES  Activity  \
59  COc1ccc(C(=O)NCCNC(=O)c2cnccn2)cc1OC         0   

                                            Molecule  \
59  <rdkit.Chem.rdchem.Mol object at 0x7fb8f320af80>   

                        Canonical_SMILES  
59  COc1ccc(C(=O)NCCNC(=O)c2cnccn2)cc1OC  


This molecule has the activity of 0 and 1 which is missleading and therefore should be removed. We remove rows from the DataFrame where the index corresponds to duplicate values in the 'Canonical_SMILES' column and then create a new DataFrame. The new DataFrame has 3286 unique compounds in total, with xx active and xx inactive.

In [7]:
df_cleaned = df.loc[~df.index.isin(duplicates_indices)].reset_index(drop=True)
df_cleaned.shape

(3286, 4)

We create a new column in Pandas DataFrame to represent molecular objects (Mol) from Canonical SMILES.

In [8]:
df_cleaned['Canonical_Mol']= [Chem.MolFromSmiles(s) for s in df_cleaned['Canonical_SMILES']]
df_cleaned.head(2)

,SMILES,Activity,Molecule,Canonical_SMILES,Canonical_Mol
0,COc1cc(Cl)cc(/C=N\NC(=O)c2ccc3c(c2)OCO3)c1O,1,<rdkit.Chem.rdchem.Mol object at 0x7fb8f31f2d50>,COc1cc(Cl)cc(/C=N\NC(=O)c2ccc3c(c2)OCO3)c1O,<rdkit.Chem.rdchem.Mol object at 0x7fb8f372e440>
1,CCOC(=O)CN1C(=O)S/C(=C/c2ccc(OCc3ccc(C(=O)O)cc...,1,<rdkit.Chem.rdchem.Mol object at 0x7fb8f31f2d00>,CCOC(=O)CN1C(=O)S/C(=C/c2ccc(OCc3ccc(C(=O)O)cc...,<rdkit.Chem.rdchem.Mol object at 0x7fb8f372e3f0>


We can use SaltRemover library to remove any salt from the canonical SMILES. Here, we make a new function, "remove_salts_from_smiles", that takes a list of SMILES  as input, iterates through the list, and uses the RDKit SaltRemover to remove salts from each SMILES string. It then adds the cleaned SMILES strings to a new list, which is returned as the result. The new list of SMILES, "DESALTSMILES".

In [9]:
remover = SaltRemover.SaltRemover()
mols = [Chem.MolFromSmiles(smi) for smi in df_cleaned['Canonical_SMILES']]
df_cleaned['DESALTSMILES'] = [Chem.MolToSmiles(remover.StripMol(s)) for s in mols]

The DESALTSMILES then can be converted to molecular objects (Mol) for molecular visualisation.

In [10]:
df_cleaned['DESALT_Molecules']= [Chem.MolFromSmiles(s) for s in df_cleaned["DESALTSMILES"]]

As we might have removed cationic/anionic fragments from canonical SMILES, we might create molecules with positive or negative charges. Thus, we need to neutralise molecules which is a common task in cheminformatics, especially when working with molecules from different sources where the protonation states might not be consistent. Neutralization typically involves adding or removing protons (H+ ions) from the molecule to achieve a neutral charge. Here we create a Python function that can neutralise molecules.

In [11]:
def neutralize_molecule(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol:
        charge = Chem.GetFormalCharge(mol)
        if charge != 0:
            mol = AllChem.AddHs(mol)
            Chem.GetFormalCharge(mol)
            mol = Chem.RemoveHs(mol)
        return Chem.MolToSmiles(mol)
    else:
        return None
df_cleaned["NEU_SMILES"] = df_cleaned["DESALTSMILES"].apply(neutralize_molecule)
df_cleaned.head(2)


,SMILES,Activity,Molecule,Canonical_SMILES,Canonical_Mol,DESALTSMILES,DESALT_Molecules,NEU_SMILES
0,COc1cc(Cl)cc(/C=N\NC(=O)c2ccc3c(c2)OCO3)c1O,1,<rdkit.Chem.rdchem.Mol object at 0x7fb8f31f2d50>,COc1cc(Cl)cc(/C=N\NC(=O)c2ccc3c(c2)OCO3)c1O,<rdkit.Chem.rdchem.Mol object at 0x7fb8f372e440>,COc1cc(Cl)cc(/C=N\NC(=O)c2ccc3c(c2)OCO3)c1O,<rdkit.Chem.rdchem.Mol object at 0x7fb8f3797e90>,COc1cc(Cl)cc(/C=N\NC(=O)c2ccc3c(c2)OCO3)c1O
1,CCOC(=O)CN1C(=O)S/C(=C/c2ccc(OCc3ccc(C(=O)O)cc...,1,<rdkit.Chem.rdchem.Mol object at 0x7fb8f31f2d00>,CCOC(=O)CN1C(=O)S/C(=C/c2ccc(OCc3ccc(C(=O)O)cc...,<rdkit.Chem.rdchem.Mol object at 0x7fb8f372e3f0>,CCOC(=O)CN1C(=O)S/C(=C/c2ccc(OCc3ccc(C(=O)O)cc...,<rdkit.Chem.rdchem.Mol object at 0x7fb8f3797800>,CCOC(=O)CN1C(=O)S/C(=C/c2ccc(OCc3ccc(C(=O)O)cc...


We have cleaned and processed our dataset. 
Herein, we are going to create different function to calculate MACCs key and Morgan FPs using the pre-processed data.
1. Morgan FPs:

In [12]:
def morgan_fpts(smiles):
    Morgan_fpts = []
    for i in smiles:
        mol = Chem.MolFromSmiles(i)
        fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,2,256)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)
    return np.array(Morgan_fpts)
Morgan_fpts = morgan_fpts(df_cleaned['DESALTSMILES'])
Morgan_fingerprints = pd.DataFrame(Morgan_fpts,columns=['Bit{}'.format(i) for i in range(Morgan_fpts.shape[1])])
Morgan_fingerprints.head(2)

,Bit0,Bit1,Bit2,Bit3,Bit4,Bit5,Bit6,Bit7,Bit8,Bit9,...,Bit246,Bit247,Bit248,Bit249,Bit250,Bit251,Bit252,Bit253,Bit254,Bit255
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


2. MACCs Keys:

In [13]:
def maccs_fpts(smiles):
  maccs_fpts = []
  mols = [Chem.MolFromSmiles(smi) for smi in smiles]
  maccs_keys = [MACCSkeys.GenMACCSKeys(m) for m in mols]
  mcfps = np.array(maccs_keys)
  return mcfps
maccs_fpts_array = maccs_fpts(df_cleaned['DESALTSMILES'])
maccs_fingerprints = pd.DataFrame(maccs_fpts_array,columns=['Bit{}'.format(i) for i in range(maccs_fpts_array.shape[1])])
maccs_fingerprints.head(2)

,Bit0,Bit1,Bit2,Bit3,Bit4,Bit5,Bit6,Bit7,Bit8,Bit9,...,Bit157,Bit158,Bit159,Bit160,Bit161,Bit162,Bit163,Bit164,Bit165,Bit166
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


And now we calculate other molecular descriptors such as physchem proerties:

In [14]:
#calculate molecular descriptors
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names
Mol_descriptors,desc_names = RDkit_descriptors(df_cleaned['DESALTSMILES'])
df_cleaned_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_cleaned_descriptors.head(2)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.845841,-3.164643,12.845841,0.315478,0.654394,348.742,335.638,348.051299,124,0,...,0,0,0,0,0,0,0,0,0,0
1,13.110162,-3.870251,13.110162,0.221533,0.489076,441.461,422.309,441.088223,160,0,...,1,0,0,0,0,0,0,0,0,0


We are going to create two different datasets by merging Morgan FPs MACCs and MACCs keys with molecular descriptors. \so, in total, we will have 4 datasets: 
1. Morgan FPs
2. MACCS FPs
3. Morgan FPs merged with other descriptors (e.g., Physchem )
4. MACCS FPs merged with other descriptors (e.g., Physchem )


In [173]:
dataset_3 = pd.concat([Morgan_fingerprints, df_cleaned_descriptors], axis=1).clip(upper=1e15)
dataset_4 = pd.concat([maccs_fingerprints, df_cleaned_descriptors], axis=1).clip(upper=1e15)

Merging data sets

In [55]:
#Model 1 with Morgan FPs and RF
X=Morgan_fingerprints.copy()
y=df_cleaned['Activity']
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size= 0.2, random_state=0, stratify=y)
classifier = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
rf= RandomForestClassifier()
rf.fit(X_train,y_train)
predictions= rf.predict(X_test)
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[305  24]
 [ 24 305]]


              precision    recall  f1-score   support

           0       0.93      0.93      0.93       329
           1       0.93      0.93      0.93       329

    accuracy                           0.93       658
   macro avg       0.93      0.93      0.93       658
weighted avg       0.93      0.93      0.93       658



In [18]:
#Model 2 with MACCs FPs and RF
X=maccs_fingerprints.copy()
y=df_cleaned['Activity']  # Tox Class
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size= 0.2, random_state=0, stratify=y)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
rf= RandomForestClassifier()
rf.fit(X_train,y_train)
predictions= rf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[312  17]
 [ 22 307]]


              precision    recall  f1-score   support

           0       0.93      0.95      0.94       329
           1       0.95      0.93      0.94       329

    accuracy                           0.94       658
   macro avg       0.94      0.94      0.94       658
weighted avg       0.94      0.94      0.94       658



In [176]:
#Model 3 with Morgan FPs and RDKit Descriptors and RF Model
X=dataset_3.copy()
y=df_cleaned['Activity']
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size= 0.2, random_state=0, stratify=y)
classifier = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
rf= RandomForestClassifier()
rf.fit(X_train,y_train)
predictions= rf.predict(X_test)
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[313  16]
 [ 24 305]]


              precision    recall  f1-score   support

           0       0.93      0.95      0.94       329
           1       0.95      0.93      0.94       329

    accuracy                           0.94       658
   macro avg       0.94      0.94      0.94       658
weighted avg       0.94      0.94      0.94       658



In [177]:
#Model 4 with MACCs FPs and RDKit Descriptors and RF Model
X=dataset_3.copy()
y=df_cleaned['Activity']
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size= 0.2, random_state=0, stratify=y)
classifier = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
rf= RandomForestClassifier()
rf.fit(X_train,y_train)
predictions= rf.predict(X_test)
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[314  15]
 [ 28 301]]


              precision    recall  f1-score   support

           0       0.92      0.95      0.94       329
           1       0.95      0.91      0.93       329

    accuracy                           0.93       658
   macro avg       0.94      0.93      0.93       658
weighted avg       0.94      0.93      0.93       658

